In [8]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

import torch
from transformers import DistilBertModel, DistilBertTokenizer
from src.dataset import UnsplashDataset
from torch.utils.data import DataLoader
from torch import nn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
dataset = UnsplashDataset("../../data/unsplash/photos.tsv*")
train_loader = DataLoader(dataset, batch_size = 4, shuffle=True)    
img, label = next(iter(train_loader))

In [10]:
class TextEncoder(nn.Module):
    def __init__(self, model_name="distilbert-base-uncased", pretrained=True, trainable=True):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name)
            
        for p in self.model.parameters():
            p.requires_grad = trainable

        # we are using the CLS token hidden representation as the sentence's embedding
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]

In [11]:
text_model = TextEncoder()

In [12]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
data = tokenizer(label, padding=True, truncation=True, max_length = 76, return_tensors="pt")

In [25]:
text_model(data['input_ids'], data['attention_mask'])

torch.Size([4, 768])